In [1]:
import os
import sys
import django
import logging
import itertools
import pandas as pd
import numpy as np
from datetime import datetime
from tqdm import tqdm
from django.db.models import Count
from sklearn.metrics import mean_squared_error
sys.path.insert(0, "/Users/victornguyen/Sites/07.book_management/")
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "book_management.settings")
django.setup()
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.DEBUG)
import django
import time
from main_site.models import Rating
from title.models import Book
from transaction.models import Master, Detail
import rs_system.neighborhood_cf_rs as nb_rs

logger = logging.getLogger('Evaluation')

In [2]:
transaction_count = Detail.objects.all().count()
test_data_count = round(transaction_count * 10 /100)
book_df = pd.DataFrame(list(Book.objects.all().values()))
# Change transaction_master_db to data frame
transaction_master_df = pd.DataFrame(list(Master.objects.all().values()))
# Change name column
transaction_master_df.rename(columns={'date': 'hire_date'},
                             inplace=True)
# Change transaction_detail_db to data frame
transaction_detail_df = pd.DataFrame(list(Detail.objects.all().values()))
# Join two data frame to one
transaction_df = pd.merge(transaction_master_df, transaction_detail_df, left_on='id', right_on='transaction_id')
transaction_df = pd.merge(transaction_df, book_df[['barcode', 'title_id']], left_on="book_id",
                          right_on="barcode",
                          how="left")
transaction_df = transaction_df.drop(['id_x', 'transaction_id'], axis=1)
transaction_df['return_date'] = transaction_df['return_date'].fillna(datetime.now())
transaction_count_df = transaction_df[['title_id','user_id']].groupby('user_id').agg('count')
transaction_count_df = transaction_count_df.sort_values(by=['title_id'], ascending=False)
transaction_count_df.rename(columns={'title_id': 'count_title_id'},
                                     inplace=True)
transaction_count_df
total_transaction_df = pd.merge(transaction_df, transaction_count_df, left_on="user_id",
                          right_on="user_id")
total_transaction_df = total_transaction_df.sort_values(by=['count_title_id','user_id','hire_date'], ascending=False).reset_index(drop=True)
total_transaction_df.head(10)

,hire_date,user_id,book_id,due_date,hire_type,id_y,price,return_date,status,barcode,title_id,count_title_id
0,2018-01-29,48,53397-70,2018-07-15,2,93,62620.00,2018-06-08 11:37:12,1,53397-70,55,8
1,2017-10-02,48,58601-80,2018-02-15,2,65,69488.00,2018-01-23 09:39:45,1,58601-80,21,8
2,2017-09-25,48,58604-80,2018-02-15,2,40,100000.00,2018-01-23 09:39:54,1,58604-80,103,8
3,2017-09-25,48,58598-130,2018-02-15,2,41,74538.00,2018-01-23 09:39:38,1,58598-130,56,8
4,2017-09-25,48,58602-80,2018-02-15,2,42,63327.00,2018-01-23 09:39:27,1,58602-80,41,8
5,2017-09-18,48,58604-80,2017-09-25,1,34,25000.00,2017-09-20 09:09:05,1,58604-80,103,8
6,2017-09-18,48,58598-130,2017-09-25,1,35,18634.50,2017-09-20 09:09:23,1,58598-130,56,8
7,2017-09-18,48,58602-80,2017-09-25,1,36,15831.75,2017-09-20 09:08:36,1,58602-80,41,8
8,2018-09-17,39,58585-80,2019-02-15,2,139,100000.00,2019-01-14 15:20:48,1,58585-80,119,8
9,2018-09-15,39,53698-80,2019-02-15,2,138,46864.00,2019-01-21 09:28:30,1,53698-80,12,8


In [3]:
test_user_tile = list()
count = 0
current_row = 0
remove_row = []
while True:
    total_transaction_df
    row = total_transaction_df.iloc[current_row]
    if row.count_title_id == 1:
        break
    if row.count_title_id > 10:
        limit_row = 3
    elif row.count_title_id > 5:
        limit_row = 2
    else:
        limit_row = 1
    for i in range(limit_row):
        remove_row.append(current_row+i)
    title_list = []
    test_user_tile.append(dict([('user_id', row.user_id), ('title_id', list(total_transaction_df.iloc[current_row:current_row+limit_row]['title_id']))]))
    current_row = current_row + row.count_title_id
    count += limit_row
    if count == test_data_count:
        break
print(remove_row)
print(test_user_tile)
train_transaction_df = total_transaction_df.drop(remove_row).reset_index(drop=True)
train_transaction_df.head()

[0, 1, 8, 9, 16, 17, 22, 27, 31, 35, 39, 43, 46, 49, 52, 55, 58, 61, 64]
[{'user_id': 48, 'title_id': [55, 21]}, {'user_id': 39, 'title_id': [119, 12]}, {'user_id': 45, 'title_id': [106, 130]}, {'user_id': 122, 'title_id': [115]}, {'user_id': 62, 'title_id': [101]}, {'user_id': 28, 'title_id': [54]}, {'user_id': 26, 'title_id': [102]}, {'user_id': 8, 'title_id': [140]}, {'user_id': 114, 'title_id': [150]}, {'user_id': 113, 'title_id': [20]}, {'user_id': 89, 'title_id': [129]}, {'user_id': 75, 'title_id': [101]}, {'user_id': 68, 'title_id': [98]}, {'user_id': 47, 'title_id': [130]}, {'user_id': 23, 'title_id': [6]}, {'user_id': 16, 'title_id': [88]}]


,hire_date,user_id,book_id,due_date,hire_type,id_y,price,return_date,status,barcode,title_id,count_title_id
0,2017-09-25,48,58604-80,2018-02-15,2,40,100000.0,2018-01-23 09:39:54,1,58604-80,103,8
1,2017-09-25,48,58598-130,2018-02-15,2,41,74538.0,2018-01-23 09:39:38,1,58598-130,56,8
2,2017-09-25,48,58602-80,2018-02-15,2,42,63327.0,2018-01-23 09:39:27,1,58602-80,41,8
3,2017-09-18,48,58604-80,2017-09-25,1,34,25000.0,2017-09-20 09:09:05,1,58604-80,103,8
4,2017-09-18,48,58598-130,2017-09-25,1,35,18634.5,2017-09-20 09:09:23,1,58598-130,56,8


In [4]:
def cal_point(sample):
    hire_date_length = abs(sample.return_date - sample.hire_date)
    due_date_length = abs(sample.due_date - sample.hire_date)
    ratio = hire_date_length / due_date_length
    sample['hire_date_length'] = hire_date_length
    sample['due_date_length'] = due_date_length
    sample['ratio_hire_date'] = round(ratio, 3)
    if (ratio > 2).bool():
        sample['point'] = 4.50
    elif (ratio > 1).bool():
        ratio_overdue = abs(1 - ratio)
        sample['point'] = 3 + (7 - (7 * ratio_overdue))
    else:
        sample['test'] = abs((1 - ratio) / (ratio - 1))
        sample['point'] = 3 + (7 * ratio)
    sample['point'] = round(sample['point'], 2)
    return sample

In [5]:
transaction_df = transaction_df.groupby('id_y').apply(cal_point).reset_index(drop=True)

# New data frame with user_id, title_id, point
train_data_df = pd.DataFrame()
train_data_df['user_id'] = transaction_df.user_id
train_data_df['title_id'] = transaction_df.title_id
train_data_df['rating'] = transaction_df.point
train_data_df = train_data_df.groupby(['user_id', 'title_id']).mean().round(3).reset_index()
train_data_df.head()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/groupby/groupby.py:887: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  result = concat(values, axis=self.axis)


,user_id,title_id,rating
0,8,20,9.430
1,8,119,9.430
2,8,140,4.325
3,9,113,9.080
4,10,14,9.960


In [6]:
y_true = [3, -0.5, 2, 7]
y_pred = [2.5, 0.0, 2, 8]
mean_squared_error(y_true, y_pred)

0.375

In [7]:
res_nb = nb_rs.RecommendationNB()
for item in tqdm(test_user_tile):
    count +=1
    book_id_list = []
    rec_list = res_nb.get_list_recommendation(item['user_id'])
    for i in rec_list:
        book_id_list.append(int(i[0]))
    print('user',item['user_id'])
    print(item['title_id'])
    print('predict:',book_id_list)
    print(list(set(item['title_id']) & set(book_id_list)))
    if len(book_id_list) != 0:
        error = len(list(set(item['title_id']) & set(book_id_list)))/len(book_id_list)
        print(error)
#     y_true.append(float(row.rating))
#     predict_result = res_nb.predict(row.user_id, row.title_id, rating_matrix, mean_centered_rating_matrix, user_similarity_matrix)
#     y_pred.append(predict_result)

  0%|          | 0/16 [00:00<?, ?it/s]2019-07-02 00:53:27,980 : INFO : Load all rating data
2019-07-02 00:53:27,999 : DEBUG : Creating ratings matrix

0it [00:00, ?it/s]
145it [00:00, 18183.76it/s]--- Logging error ---
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/logging/__init__.py", line 993, in emit
    msg = self.format(record)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/logging/__init__.py", line 839, in format
    return fmt.format(record)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/logging/__init__.py", line 576, in format
    record.message = record.getMessage()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/logging/__init__.py", line 338, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", li

145it [00:00, 37868.87it/s]--- Logging error ---
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/logging/__init__.py", line 993, in emit
    msg = self.format(record)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/logging/__init__.py", line 839, in format
    return fmt.format(record)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/logging/__init__.py", line 576, in format
    record.message = record.getMessage()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/logging/__init__.py", line 338, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 85, in _run_code
    exec

user 48
[55, 21]
predict: []
[]



 12%|█▎        | 2/16 [00:11<01:19,  5.65s/it]t/s]2019-07-02 00:53:39,323 : INFO : Load all rating data
2019-07-02 00:53:39,333 : DEBUG : Creating ratings matrix

0it [00:00, ?it/s]
145it [00:00, 43142.09it/s]--- Logging error ---
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/logging/__init__.py", line 993, in emit
    msg = self.format(record)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/logging/__init__.py", line 839, in format
    return fmt.format(record)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/logging/__init__.py", line 576, in format
    record.message = record.getMessage()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/logging/__init__.py", line 338, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/

user 39
[119, 12]
predict: [119, 115, 11, 87, 99, 129, 105, 22]
[119]
0.125


...

user 47
[130]
predict: []
[]



 94%|█████████▍| 15/16 [01:41<00:06,  6.76s/it]s] 2019-07-02 00:55:09,597 : INFO : Load all rating data
2019-07-02 00:55:09,614 : DEBUG : Creating ratings matrix

0it [00:00, ?it/s]
145it [00:00, 54408.13it/s]--- Logging error ---
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/logging/__init__.py", line 993, in emit
    msg = self.format(record)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/logging/__init__.py", line 839, in format
    return fmt.format(record)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/logging/__init__.py", line 576, in format
    record.message = record.getMessage()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/logging/__init__.py", line 338, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/

user 23
[6]
predict: [149]
[]
0.0



100%|██████████| 16/16 [01:47<00:00,  6.53s/it]s]

user 16
[88]
predict: []
[]
